<a href="https://colab.research.google.com/github/vruddhis/deepLearning/blob/main/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil


url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb_v1')

os.listdir(dataset_dir)

train_dir = os.path.join(dataset_dir, 'aclImdb', 'train')
test_dir = os.path.join(dataset_dir, 'aclImdb', 'test')
os.listdir(train_dir)

['urls_neg.txt',
 'urls_pos.txt',
 'unsupBow.feat',
 'labeledBow.feat',
 'unsup',
 'neg',
 'urls_unsup.txt',
 'pos']

In [ ]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [ ]:

import tensorflow_text as text
import re
import string


def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')


vectorize_layer = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=10000, #idk i just copied
    output_mode='int',
    output_sequence_length=250)


train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)



In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import losses
import tensorflow as tf

VOCAB_SIZE = 10000 #what i used upar

model = tf.keras.Sequential([
    layers.Embedding(VOCAB_SIZE, 16), #embedding layer
    layers.GlobalAveragePooling1D(),
    layers.Dense(10, activation='relu'),
    layers.Dense(10, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])


model.compile(loss=losses.BinaryCrossentropy(),
              optimizer='adam',
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.5)])

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - binary_accuracy: 0.6033 - loss: 0.6532 - val_binary_accuracy: 0.8514 - val_loss: 0.3742
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - binary_accuracy: 0.8528 - loss: 0.3514 - val_binary_accuracy: 0.8564 - val_loss: 0.3283
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - binary_accuracy: 0.8966 - loss: 0.2623 - val_binary_accuracy: 0.8792 - val_loss: 0.2923
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - binary_accuracy: 0.9093 - loss: 0.2310 - val_binary_accuracy: 0.8792 - val_loss: 0.2924
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - binary_accuracy: 0.9270 - loss: 0.1950 - val_binary_accuracy: 0.8808 - val_loss: 0.2949
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - binary_accuracy: 0.9261 - loss: 0.1874 - val_binary_accuracy: 0.8770 - val_loss: 0.3230
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - binary_accuracy: 0.9470 - loss: 0.1518 - val_binary_accuracy: 0.8716 - val_loss: 0.3343
Epoch 

In [ ]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - binary_accuracy: 0.8552 - loss: 0.4294
Loss:  0.42756715416908264
Accuracy:  0.8551599979400635


In [ ]:
import numpy as np

learning_rates = [0.001, 0.0005, 0.0001]
dense_1_neurons = [10, 20, 30]
dense_2_neurons = [10, 20, 30]

best_accuracy = 0
best_hyperparameters = {}

for lr in learning_rates:
    for d1_neurons in dense_1_neurons:
        for d2_neurons in dense_2_neurons:
            print(f"Training with Learning Rate: {lr}, Dense 1 Neurons: {d1_neurons}, Dense 2 Neurons: {d2_neurons}")

            model = tf.keras.Sequential([
                layers.Embedding(VOCAB_SIZE, 16),
                layers.GlobalAveragePooling1D(),
                layers.Dense(d1_neurons, activation='relu'),
                layers.Dense(d2_neurons, activation='relu'),
                layers.Dense(1, activation='sigmoid')
            ])

            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                          loss=losses.BinaryCrossentropy(),
                          metrics=['accuracy'])

            history = model.fit(
                train_ds,
                epochs=5,
                validation_data=test_ds
            )

            loss, accuracy = model.evaluate(test_ds, verbose=0)
            print(f"Test Accuracy: {accuracy}")

            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_hyperparameters = {
                    'learning_rate': lr,
                    'dense_1_neurons': d1_neurons,
                    'dense_2_neurons': d2_neurons
                }


print("\nRandom Search Complete.")
print(f"Best Accuracy: {best_accuracy}")
print(f"Best Hyperparameters: {best_hyperparameters}")


Training with Learning Rate: 0.001, Dense 1 Neurons: 10, Dense 2 Neurons: 10
Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.5875 - loss: 0.6632 - val_accuracy: 0.7950 - val_loss: 0.4382
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.8429 - loss: 0.3634 - val_accuracy: 0.8420 - val_loss: 0.3517
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.8800 - loss: 0.2846 - val_accuracy: 0.8598 - val_loss: 0.3285
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9104 - loss: 0.2280 - val_accuracy: 0.8685 - val_loss: 0.3179
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.9225 - loss: 0.2015 - val_accuracy: 0.8660 - val_loss: 0.3293
Test Accuracy: 0.8660399913787842
Training with Learning Rate: 0.001, Dense 1 Neurons: 10, Dense 2 Neurons: 20
Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.5977 - loss: 0.6516 - val_accuracy: 0.8028 - val_loss: 0.4123
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s